TO CREATE THE SPARK SESSION

In [1]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.master("local[1]").appName("Assignment1.com").getOrCreate()


24/09/26 11:28:20 WARN Utils: Your hostname, AI-CJB-LAP-459 resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface wlp0s20f3)
24/09/26 11:28:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/26 11:28:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


LOADING THE CSV FILE AS AN RDD  AND CONVERTING RDD TO DATAFRAME

In [2]:
data=spark.read.options(delimiter=",",header="true").csv("/home/ai/Downloads/assignment/hotels.csv")
rdd1=data.rdd
print("type:",type(rdd1))
df=rdd1.toDF()
df.show(1,truncate=False)

type: <class 'pyspark.rdd.RDD'>


+---------------+--------------+---------+----------------+----------+----------+--------+--------------+------------------------------+
|Holding company|Headquarters  |Locations|Year established|Properties|Rooms     |Category|Managed brands|Franchised brands             |
+---------------+--------------+---------+----------------+----------+----------+--------+--------------+------------------------------+
|Accor          |Paris (France)|Worldwide|1967            |5,300[4]  |778,000[4]|Luxury  |Raffles       |Banyan Tree Hótels and Resorts|
+---------------+--------------+---------+----------------+----------+----------+--------+--------------+------------------------------+
only showing top 1 row



TO CREATE A DATAFRAME WHERE IT CONTAINS  COLUMNS["Holding company", "Headquarters", "Locations", "Year established", "Properties", "Rooms"]

In [3]:
df1 = df.select("Holding company","Headquarters", "Locations", "Year established", "Properties", "Rooms")


only 1 row per "Holding company", be sure to concatenate info if they appear in more than one row/cell)

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


group_df=df.groupBy("Holding company").agg(
     concat_ws(", ", collect_list(col("Headquarters"))).alias("Headquarters"),
    concat_ws(", ", collect_list(col("Locations"))).alias("Locations"),
    min(col("Year established")).alias("Year established"),
    sum(col("Properties")).alias("Properties"),
    sum(col("Rooms")).alias("Rooms"))

group_df.show(5,truncate=False)

+----------------------------+---------------------------------+---------+----------------+----------+-----+
|Holding company             |Headquarters                     |Locations|Year established|Properties|Rooms|
+----------------------------+---------------------------------+---------+----------------+----------+-----+
|Barrière                    |Paris (France)                   |Europe   |1912[11]        |NULL      |NULL |
|Centara Hotels & Resorts[18]|Bangkok (Thailand)               |Asia     |1983[19]        |NULL      |NULL |
|Hoshino Resorts[50]         |Karuizawa (Japan)                |Japan    |1904            |32.0      |NULL |
|Lifestyle Holidays Group    |Puerto Plata (Dominican Republic)|Caribbean|2002            |10.0      |NULL |
|Red Roof Inn                |New Albany, Ohio (USA)           |USA      |1973[103]       |NULL      |NULL |
+----------------------------+---------------------------------+---------+----------------+----------+-----+
only showing top 5 

NO MISSING IN ANY COLUMNS
imputing the null values with mode


CLEAN INFORMATION ABOUT THE REFERENCES (THEY APPEAR AS[NUMBER])


In [5]:
group_df.printSchema()

root
 |-- Holding company: string (nullable = true)
 |-- Headquarters: string (nullable = false)
 |-- Locations: string (nullable = false)
 |-- Year established: string (nullable = true)
 |-- Properties: double (nullable = true)
 |-- Rooms: double (nullable = true)



In [6]:
print(group_df.columns)

['Holding company', 'Headquarters', 'Locations', 'Year established', 'Properties', 'Rooms']


In [7]:
from pyspark.ml.feature import Imputer

group_df.filter(col("Year established").isNull()).show(5)
group_df = group_df.withColumn("Year established", col("Year established").cast(IntegerType()))

imputer = Imputer(
    inputCols=["Year established", "Properties", "Rooms"],
    outputCols=["Year established", "Properties", "Rooms"],
    strategy="mode"
)
print("DataFrame imputed with mode")

res_df = imputer.fit(group_df).transform(group_df)
res_df.show(5, truncate=False)



+--------------------+------------------+---------+----------------+----------+-----+
|     Holding company|      Headquarters|Locations|Year established|Properties|Rooms|
+--------------------+------------------+---------+----------------+----------+-----+
|Pan Pacific Hotel...|     Singapore[97]|     Asia|            NULL|      NULL| NULL|
| Huazhu Hotels Group|                  |         |            NULL|      NULL| NULL|
|Walt Disney Parks...|     Burbank (USA)|         |            NULL|      NULL| NULL|
| Compass Hospitality|Bangkok (Thailand)| Thailand|            NULL|      NULL| NULL|
+--------------------+------------------+---------+----------------+----------+-----+

DataFrame imputed with mode
+----------------------------+---------------------------------+---------+----------------+----------+-----+
|Holding company             |Headquarters                     |Locations|Year established|Properties|Rooms|
+----------------------------+---------------------------------+-

CREATE A NEW COLUMN ASSOCIATED "HEADQUARTERS COUNTRY"(NOW IS PART OF THE COLUMN "HEADQUARTERS")

In [8]:
from pyspark.sql import functions as F

res_df = res_df.withColumn(
    "Headquarters country",
    F.regexp_extract(col("Headquarters"), r'\(([^)]+)\)', 1)  # Extract text within parentheses
)
# to remove the country from the headquarters
res_df = res_df.withColumn(
    "Headquarters",
    F.regexp_replace(col("Headquarters"), r'\s*\(.*\)', '')  # Remove the text in parentheses
)

res_df.show(5,truncate=False)
# Show the result with the new column
res_df.select("Headquarters", "Headquarters country").show(5,truncate=False)

+----------------------------+-----------------+---------+----------------+----------+-----+--------------------+
|Holding company             |Headquarters     |Locations|Year established|Properties|Rooms|Headquarters country|
+----------------------------+-----------------+---------+----------------+----------+-----+--------------------+
|Barrière                    |Paris            |Europe   |1988            |10.0      |766.0|France              |
|Centara Hotels & Resorts[18]|Bangkok          |Asia     |1988            |10.0      |766.0|Thailand            |
|Hoshino Resorts[50]         |Karuizawa        |Japan    |1904            |32.0      |766.0|Japan               |
|Lifestyle Holidays Group    |Puerto Plata     |Caribbean|2002            |10.0      |766.0|Dominican Republic  |
|Red Roof Inn                |New Albany, Ohio |USA      |1988            |10.0      |766.0|USA                 |
+----------------------------+-----------------+---------+----------------+----------+--

SECOND DATAFRAME

THE SECOND DATAFRAME WITH COLUMNSA["HOLDING COMPANY","CATEGORY","MANAGED BRANDS","FRANCHISED BRANDS"]

In [10]:
df2 = df.select(
    "Holding company",
    "Category",           # This column should already exist in df
    "Managed brands",     # This column should already exist in df
    "Franchised brands"   # This column should already exist in df
)
df2.show(1, truncate=False)


+---------------+--------+--------------+------------------------------+
|Holding company|Category|Managed brands|Franchised brands             |
+---------------+--------+--------------+------------------------------+
|Accor          |Luxury  |Raffles       |Banyan Tree Hótels and Resorts|
+---------------+--------+--------------+------------------------------+
only showing top 1 row



ONLY THE FRANCHISED BRANDS COLUMN CAN POSSESS NULL VALUES

In [19]:
#droping out the rows with null values
filtered_df = df2.filter(F.col("Holding company").isNotNull())

# Show the filtered DataFrame
filtered_df.show(5,truncate=False)

#imputing the category column with mode value
mode_value = filtered_df.groupBy("Category").count().orderBy(F.desc("count")).first()[0]

# Step 2: Fill nulls in the "Category" column with the mode value
imputed_df = filtered_df.fillna({"Category": mode_value})

# Show the final DataFrame
imputed_df.show(5,truncate=False)

+-------------------------+--------+------------------------------------------------------------------------------------------------------------------------+------------------------------+
|Holding company          |Category|Managed brands                                                                                                          |Franchised brands             |
+-------------------------+--------+------------------------------------------------------------------------------------------------------------------------+------------------------------+
|Accor                    |Luxury  |Raffles                                                                                                                 |Banyan Tree Hótels and Resorts|
|Aman Resorts             |Luxury  |Aman                                                                                                                    |NULL                          |
|APA Group                |Midscale|Coast Hotels       

SPECIFYING THE BEST TYPE AND NULLABLE FOR EACH NCOLUMN

In [12]:
# Define the schema with data types and nullability
schema = StructType([
    StructField("Holding company", StringType(), nullable=True),
    StructField("Category", StringType(), nullable=True),  # Assuming it's nullable
    StructField("Managed brands", StringType(), nullable=True),  # Assuming it's nullable
    StructField("Franchised brands", StringType(), nullable=True)  # Assuming it's nullable
])

# Create or modify the DataFrame with the defined schema
df_with_schema = spark.createDataFrame(imputed_df.rdd, schema)

# Show the schema
df_with_schema.printSchema()

root
 |-- Holding company: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Managed brands: string (nullable = true)
 |-- Franchised brands: string (nullable = true)



USE PYSPARK DATAFRAME GROUPBY TO COMPUTE

  TOTAL NUMBER OF ROOMS PER "HEADQUARTERD COUNTRY"

  AVERAGE NUMBER OF PROPERTIES PER "HEADQUARTERED COUNTRY"

  NUMBER OF "MANAGED BRANDS" PER "HOLDING COMPANY" AND "CATEGORY"

In [13]:
print(res_df.columns)

['Holding company', 'Headquarters', 'Locations', 'Year established', 'Properties', 'Rooms', 'Headquarters country']


In [14]:
rooms_per_country = res_df.groupBy("Headquarters country").agg(
    F.sum("Rooms").alias("Total Rooms")
)

# Average number of properties per "Headquarters country"
avg_properties_per_country = res_df.groupBy("Headquarters country").agg(
    F.avg("Properties").alias("Average Properties")
)

# Show results
rooms_per_country.show(5,truncate=False)
avg_properties_per_country.show(5,truncate=False)

+--------------------+-----------+
|Headquarters country|Total Rooms|
+--------------------+-----------+
|Sweden              |3064.0     |
|Germany             |766.0      |
|France              |1532.0     |
|Sri Lanka           |1532.0     |
|China               |3064.0     |
+--------------------+-----------+
only showing top 5 rows

+--------------------+------------------+
|Headquarters country|Average Properties|
+--------------------+------------------+
|Sweden              |16.5              |
|Germany             |10.0              |
|France              |10.0              |
|Sri Lanka           |22.0              |
|China               |10.0              |
+--------------------+------------------+
only showing top 5 rows



In [18]:
# Number of "Managed Brands" per "Holding Company" and "Category"
brands_per_company_category = imputed_df.groupBy("Holding company", "Category").agg(
    F.count("Managed brands").alias("Number of Managed Brands")
)

# Show results
brands_per_company_category.show(5,truncate=False)

+---------------------------+--------+------------------------+
|Holding company            |Category|Number of Managed Brands|
+---------------------------+--------+------------------------+
|Toyoko Inn                 |Luxury  |0                       |
|Cinnamon Hotels & Resorts  |Luxury  |1                       |
|Wyndham Hotels and Resorts |Premium |1                       |
|Interstate Hotels & Resorts|Luxury  |0                       |
|China Lodging Group        |Midscale|1                       |
+---------------------------+--------+------------------------+
only showing top 5 rows



In [23]:
#The first 5 rows of the second DataFrame
df2.show(5)
#Adding row_number for rows
second_df = df2.withColumn("row_number",monotonically_increasing_id())
second_df.show()

+---------------+--------+--------------+--------------------+
|Holding company|Category|Managed brands|   Franchised brands|
+---------------+--------+--------------+--------------------+
|          Accor|  Luxury|       Raffles|Banyan Tree Hótel...|
|           NULL|    NULL|Orient Express|                NULL|
|           NULL|    NULL|        Delano|                NULL|
|           NULL|    NULL|           SO/|                NULL|
|           NULL|    NULL|Sofitel Legend|                NULL|
+---------------+--------+--------------+--------------------+
only showing top 5 rows

+---------------+--------+--------------------+--------------------+----------+
|Holding company|Category|      Managed brands|   Franchised brands|row_number|
+---------------+--------+--------------------+--------------------+----------+
|          Accor|  Luxury|             Raffles|Banyan Tree Hótel...|         0|
|           NULL|    NULL|      Orient Express|                NULL|         1|
|       

In [24]:
# Filter rows where the "Holding company" column contains string values
rows_with_string_values_df2 = second_df.filter(F.col("Holding company").isNotNull() & F.col("Holding company").cast("string").rlike("[a-zA-Z]"))

# Select row numbers
row_numbers_with_string_values_df2 = rows_with_string_values_df2.select("row_number").rdd.flatMap(lambda x: x).collect()

# Display the row numbers
print("Row numbers with string values in the Holding company column:", row_numbers_with_string_values_df2)

Row numbers with string values in the Holding company column: [0, 36, 37, 40, 41, 56, 57, 61, 65, 66, 79, 84, 91, 92, 101, 114, 115, 119, 121, 125, 130, 138, 139, 141, 142, 146, 147, 151, 152, 157, 170, 173, 177, 185, 191, 205, 207, 208, 209, 222, 223, 225, 230, 235, 236, 237, 246, 250, 251, 280, 286, 292, 294, 301, 305, 311, 313, 317, 320, 332, 342, 347, 351, 359, 369, 373, 379, 381, 383, 387, 393, 401, 405, 406, 407, 410, 414, 418, 422, 423, 424, 425, 426, 428, 447]


In [26]:
from functools import reduce
from pyspark.sql import DataFrame

In [27]:
####Divding and combining the Dataframe based on row_numbers_with_string_values_df2####
separted_df2=[]
for i in range(0,len(row_numbers_with_string_values_df2)-1):
    start=row_numbers_with_string_values_df2[i]
    final=row_numbers_with_string_values_df2[i+1]
    # print(start,final)
    # Select rows initial to final and concatenate values into a single row
    concatenated_df = second_df.filter((F.col("row_number") >= start) & (F.col("row_number") < final)) \
                    .select([F.collect_list(col).alias(col) for col in second_df.columns]) \
                    .agg(*[F.first(col).alias(col) for col in second_df.columns])
    
    separted_df2.append(concatenated_df)

# Combine DataFrames into a single DataFrame
combined_df2 = reduce(DataFrame.union, separted_df2)

# droping the row_number
combined_df2_without_column = combined_df2.drop("row_number")



In [28]:
df2_columns=combined_df2_without_column.columns
for column in df2_columns:
    # Change the data type of the column to string
    combined_df2_without_column = combined_df2_without_column.withColumn(column, F.col(column).cast("string"))

    # Remove square brackets from the string
    combined_df2_without_column = combined_df2_without_column.withColumn(column, F.regexp_replace(column, "\\[|\\]", ""))
combined_df2_without_column.show(truncate=False)

24/09/26 15:35:31 WARN DAGScheduler: Broadcasting large task binary with size 1280.2 KiB
24/09/26 15:35:31 WARN DAGScheduler: Broadcasting large task binary with size 1280.2 KiB
24/09/26 15:35:31 WARN DAGScheduler: Broadcasting large task binary with size 1280.2 KiB


+---------------------------+------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------+
|Holding company            |Category                                                                                  |Managed brands                                                                                                                                                                                                                                                                                                                                          

In [29]:
# Dropping the rows with null values and empty string "" 
combined_df2_without_column = combined_df2_without_column.dropna(subset=["Category"])

combined_df2_without_column = combined_df2_without_column.filter(F.col("Category") != "")

combined_df2_without_column = combined_df2_without_column.dropna(subset=["Holding company"])

combined_df2_without_column = combined_df2_without_column.filter(F.col("Holding company") != "")

# Dropping the rows with null values and empty string "" 

combined_df2_without_column = combined_df2_without_column.dropna(subset=["Managed brands"])

combined_df2_without_column = combined_df2_without_column.filter(F.col("Managed brands") != "")



In [30]:

# Split the "Rooms" column by commas and count the number of elements
No_Managedbrands = combined_df2_without_column.withColumn("Managed Brands", size(split(col("Managed Brands"), ","))) \
    .groupBy("Holding Company","Category") \
    .agg(sum("Managed Brands").alias("Total_Managed Brands"))

# Show the result
No_Managedbrands.show(truncate=False)

24/09/26 15:36:42 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB


+---------------------------+------------------------------------------------------------------------------------------+--------------------+
|Holding Company            |Category                                                                                  |Total_Managed Brands|
+---------------------------+------------------------------------------------------------------------------------------+--------------------+
|Accor                      |Luxury, Premium, Midscale, Economy                                                        |36                  |
|Aman Resorts               |Luxury                                                                                    |1                   |
|APA Group                  |Midscale                                                                                  |2                   |
|Ascott Limited (The)       |Luxury, Premium, Upscale, Upper-Midscale, Midscale, Lower-Midscale, Economy, Uncategorized|15                  |
|Avari

24/09/26 15:36:46 WARN DAGScheduler: Broadcasting large task binary with size 1960.2 KiB
